In [32]:
# Importa la clase SparkSession del módulo pyspark.sql.
# SparkSession es la entrada principal para usar la API de Spark SQL.
# Proporciona un entorno unificado para trabajar con estructuras de datos, ejecutar consultas SQL y más.
from pyspark.sql import SparkSession

# Importa la clase SparkContext del módulo pyspark.
# SparkContext es el núcleo de cualquier aplicación de Spark y actúa como una interfaz para acceder al cluster de Spark.
# Es responsable de la comunicación entre la aplicación y los nodos ejecutores.
from pyspark import SparkContext

-----------------------

## Configuración e Inicio de una Sesión de Spark

In [33]:
# Crear una sesión de Spark utilizando la clase SparkSession.
# La sesión de Spark es la entrada principal para trabajar con DataFrames y la API de Spark SQL.
Session_spark  = SparkSession \
          .builder \
          .appName("Demo_Spark") \
          .getOrCreate()

### Explicación del Código: Crear una Sesión de Spark

#### Detalle de cada línea:

1. **`SparkSession`:**  
   Representa el punto de entrada principal para trabajar con Apache Spark. Facilita el uso de DataFrames y la API de Spark SQL.

2. **`.builder`:**  
   Inicia la configuración para una nueva sesión de Spark. Es el lugar donde se definen las propiedades iniciales, como el nombre de la aplicación y otros ajustes opcionales.

3. **`.appName("Demo_Spark")`:**  
   Especifica un nombre para la aplicación. Este nombre se usa para identificar la aplicación en las herramientas de administración y monitorización de Spark, como su interfaz web.

4. **`.getOrCreate()`:**  
   - Si ya existe una sesión de Spark activa en el contexto, la reutiliza.  
   - Si no hay una sesión activa, crea una nueva con las configuraciones definidas previamente.

### Notas adicionales:
- Este código es esencial para inicializar el entorno de trabajo en Spark y comenzar a procesar datos.
- Configuraciones adicionales, como la asignación de memoria, recursos o la URL del cluster, pueden incluirse en la fase de construcción (`builder`).
- El nombre definido en `.appName()` es útil para identificar las aplicaciones en entornos con múltiples trabajos Spark en ejecución.

---------------------------

## Acceder a SparkContext desde SparkSession

In [34]:
SpContext = Session_spark.sparkContext

#### Detalle de la línea:

1. **`SpContext = spark.sparkContext`:**  
   - **`spark`:** Es la instancia de `SparkSession` que se ha creado previamente en el código.
   - **`sparkContext`:** Es una propiedad de la clase `SparkSession` que proporciona acceso al objeto `SparkContext`. `SparkContext` es el componente central que se utiliza para interactuar con el cluster de Spark y distribuir las tareas entre los nodos.
   - **`SpContext`:** Es la variable donde se almacena el objeto `SparkContext`, permitiendo interactuar con él para realizar operaciones en el cluster de Spark, como crear RDDs, configurar el entorno de ejecución, entre otras tareas.

#### Notas adicionales:
- Aunque `SparkContext` se puede acceder directamente a través de `SparkSession`, algunas tareas avanzadas requieren interactuar con `SparkContext` directamente. Por ejemplo, crear RDDs o trabajar con configuraciones de bajo nivel.
- En la mayoría de los casos, el acceso a `sparkContext` se realiza a través de la instancia de `SparkSession`, como se hace aquí, para facilitar el manejo de recursos y la inicialización del entorno de Spark.

------------------------

## CARGA DE DATOS

In [35]:
data = Session_spark.read.csv('cars.csv', header=True, sep=";")
data.show(5)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows



--------------------

## Conociendo la Data

In [36]:
# El método printSchema() se utiliza para imprimir la estructura del esquema del DataFrame.
# Esto incluye los nombres de las columnas, los tipos de datos asociados a cada columna y si estas pueden contener valores nulos (nullable).
# Es especialmente útil para verificar la estructura de los datos antes de realizar transformaciones o análisis.
data.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: string (nullable = true)
 |-- Cylinders: string (nullable = true)
 |-- Displacement: string (nullable = true)
 |-- Horsepower: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Origin: string (nullable = true)



## Explicación del Dataset y Plan para el ETL

### Descripción del Esquema del Dataset

El dataset contiene información de automóviles con las siguientes columnas:

- **Car** (`string`, nullable): El nombre y modelo del automóvil.
- **MPG** (`string`, nullable): Millas por galón que puede recorrer el automóvil (rendimiento).
- **Cylinders** (`string`, nullable): Número de cilindros del motor.
- **Displacement** (`string`, nullable): Cilindrada o volumen de desplazamiento del motor (generalmente en pulgadas cúbicas).
- **Horsepower** (`string`, nullable): Potencia del motor en caballos de fuerza.
- **Weight** (`string`, nullable): Peso del automóvil (posiblemente en libras).
- **Acceleration** (`string`, nullable): Tiempo que tarda en acelerar de 0 a 60 mph (en segundos).
- **Model** (`string`, nullable): Año del modelo del automóvil.
- **Origin** (`string`, nullable): Procedencia del automóvil (e.g., `US`, `Europe`, `Japan`).

#### Observaciones del Esquema
1. Todas las columnas tienen tipo `string`, lo cual no es óptimo para realizar análisis o cálculos numéricos. Necesitamos convertirlas a los tipos de datos correctos (e.g., `double` o `int`).
2. La columna `nullable` indica que puede haber valores nulos en estas columnas, por lo que debemos manejarlos durante el proceso de limpieza de datos.

### Plan para el ETL (Extract, Transform, Load)

#### 1. **Extracción**
   - Leeremos los datos desde el archivo `cars.csv` utilizando PySpark, asegurándonos de que los encabezados sean detectados correctamente.

#### 2. **Transformación**
   - **Conversión de Tipos de Datos**: 
     Convertir las columnas numéricas (`MPG`, `Cylinders`, `Displacement`, `Horsepower`, `Weight`, `Acceleration`, `Model`) a sus tipos adecuados (`double` o `int`).
   - **Manejo de Valores Nulos**: 
     Imputar valores nulos en las columnas donde sea necesario o eliminar filas con datos incompletos si no son significativos.
   - **Normalización de Datos**:
     Si fuera necesario, escalar las columnas numéricas para estandarizar los valores.
   - **Creación de Nuevas Columnas** (Opcional): 
     Añadir columnas derivadas de las existentes, como categorías basadas en `MPG` (e.g., bajo, medio, alto consumo).

#### 3. **Carga**
   - Guardar el DataFrame limpio y transformado en un formato adecuado (e.g., CSV, Parquet) para su posterior análisis.

--------------------------

### Punto 2: Transformación

#### 1. **Conversión de Tipos de Datos**
Para realizar análisis y cálculos correctos, las columnas numéricas deben ser convertidas a sus tipos adecuados:

- **Columnas a convertir:**
  - `MPG`: Rendimiento, convertir a tipo `double`.
  - `Cylinders`: Número entero de cilindros, convertir a tipo `int`.
  - `Displacement`: Cilindrada del motor, convertir a tipo `double`.
  - `Horsepower`: Potencia en caballos, convertir a tipo `double`.
  - `Weight`: Peso del vehículo, convertir a tipo `double`.
  - `Acceleration`: Tiempo de aceleración, convertir a tipo `double`.
  - `Model`: Año del modelo, convertir a tipo `int`.

**Acción**: Utilizaremos la función `cast()` de PySpark para cambiar los tipos de datos.

#### 2. **Manejo de Valores Nulos**
Los valores nulos pueden afectar el análisis. Realizaremos las siguientes acciones según la columna:
- **Estrategias:**
  - Para columnas numéricas: Sustituir los valores nulos por la media o mediana de la columna (imputación).
  - Para columnas categóricas (e.g., `Origin`): Sustituir los valores nulos con el valor más frecuente (moda).
  - Si hay demasiados valores nulos en ciertas filas, podrían eliminarse.

**Acción**: Utilizaremos funciones como `fillna()` o `dropna()` para manejar estos casos.

#### 3. **Normalización de Datos**
Estandarizar los valores de las columnas numéricas para asegurar que estén en rangos similares, lo cual es importante para modelos de Machine Learning.
- **Estrategias:**
  - Aplicar normalización Min-Max o estandarización (z-score) en columnas como `Weight`, `Horsepower` y `Acceleration`.

**Acción**: Utilizaremos funciones de PySpark como `StandardScaler` o realizaremos operaciones manuales con `withColumn`.

#### 4. **Creación de Nuevas Columnas (Opcional)**
Podemos derivar información adicional:
- **Ejemplo**: Categorizar `MPG` en niveles de consumo:
  - `MPG < 15`: Bajo consumo.
  - `15 ≤ MPG ≤ 25`: Consumo medio.
  - `MPG > 25`: Alto consumo.

**Acción**: Utilizaremos `withColumn()` y expresiones condicionales (`when` y `otherwise`) de PySpark.

-------------------------

#### 1. **Conversión de Tipos de Datos**

In [37]:
from pyspark.sql.types import DoubleType, IntegerType

# Convertir los tipos de datos de las columnas utilizando cast().
data = data \
    .withColumn("MPG", data["MPG"].cast(DoubleType())) \
    .withColumn("Cylinders", data["Cylinders"].cast(IntegerType())) \
    .withColumn("Displacement", data["Displacement"].cast(DoubleType())) \
    .withColumn("Horsepower", data["Horsepower"].cast(DoubleType())) \
    .withColumn("Weight", data["Weight"].cast(DoubleType())) \
    .withColumn("Acceleration", data["Acceleration"].cast(DoubleType())) \
    .withColumn("Model", data["Model"].cast(IntegerType()))

In [38]:
data.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)



-------------------------

#### 2. **Manejo de Valores Nulos**

1. **Contar valores nulos:**

In [39]:
from pyspark.sql.functions import col, when, count, mean

In [40]:
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+---+---------+------------+----------+------+------------+-----+------+
|Car|MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+---+---+---------+------------+----------+------+------------+-----+------+
|  0|  0|        0|           0|         0|     0|           0|    0|     0|
+---+---+---------+------------+----------+------+------------+-----+------+



2. **Imputación de valores nulos:**

In [41]:
# Aquí calculamos la media de todas las columnas numéricas (excepto las de tipo texto como "Car" y "Origin") usando `mean()`. Luego usamos `fillna()` para rellenar los valores nulos con la media correspondiente de cada columna.

mean_values = data.select([mean(c).alias(c) for c in data.columns if c != "Car" and c != "Origin"]).first()
data = data.fillna(mean_values.asDict())

3. **Eliminar filas con valores nulos en columnas clave:**

In [42]:
# Si consideramos que las columnas "MPG", "Cylinders", "Displacement", "Horsepower", "Weight", y "Acceleration" son esenciales para el análisis, podemos eliminar las filas que contienen valores nulos en cualquiera de estas columnas usando `dropna()`.

data = data.dropna(subset=["MPG", "Cylinders", "Displacement", "Horsepower", "Weight", "Acceleration"])

### Consideraciones:
- **Imputación con la media** es adecuada cuando los datos no presentan una distribución sesgada y cuando no se cuenta con un valor específico para imputar.
- **Eliminación de filas** puede ser apropiada si las filas con valores nulos son pocas o si los datos restantes son lo suficientemente representativos.

-------------------------

Para el punto de **Normalización de Datos**, existen dos estrategias comunes: **Min-Max Normalization** y **Standardization (z-score)**. A continuación, vamos a proceder con ambos enfoques utilizando PySpark.

### 1. **Normalización Min-Max (Escalado entre 0 y 1)**
La normalización Min-Max transforma los valores de las columnas numéricas a un rango [0, 1].

### 2. **Estandarización (z-score)**
La estandarización, también conocida como normalización Z-score, transforma los valores de las columnas para que tengan una media de 0 y una desviación estándar de 1.

### Código para la Normalización y Estandarización:

In [43]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

# Para normalización Min-Max, primero necesitamos convertir las columnas a un formato vectorial.
# Aplicamos la normalización Min-Max a las columnas `Weight`, `Horsepower`, y `Acceleration`

# Convertir las columnas a formato vectorial para MinMaxScaler
assembler = VectorAssembler(inputCols=["Weight", "Horsepower", "Acceleration"], outputCol="features")
data_with_features = assembler.transform(data)

# Crear el objeto MinMaxScaler y ajustarlo a los datos
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_with_features)
data_normalized = scaler_model.transform(data_with_features)

# Ahora, `data_normalized` contiene las columnas normalizadas en el campo `scaled_features`.
data_normalized.select("Car", "scaled_features").show(5)

# Para estandarización (z-score), necesitamos transformar las mismas columnas
scaler_standard = StandardScaler(inputCol="features", outputCol="standardized_features", withStd=True, withMean=True)
scaler_standard_model = scaler_standard.fit(data_with_features)
data_standardized = scaler_standard_model.transform(data_with_features)

# Mostrar resultados de la estandarización (z-score)
data_standardized.select("Car", "standardized_features").show(5)

ModuleNotFoundError: No module named 'numpy'

### Explicación del Código:

- **MinMaxScaler**:
  - **VectorAssembler**: Convierte las columnas seleccionadas en un vector para que el `MinMaxScaler` pueda operar sobre ellas.
  - **MinMaxScaler**: Normaliza los datos a un rango de [0, 1]. La columna `features` es la que contiene los datos a normalizar, y la columna `scaled_features` será el resultado.

- **StandardScaler**:
  - **StandardScaler**: Realiza una estandarización de las características, es decir, ajusta los datos de modo que tengan una media de 0 y una desviación estándar de 1.
  - Similar al `MinMaxScaler`, se crea un ensamblador de características y luego se aplica el `StandardScaler` a los datos para obtener las características estandarizadas.

### Resumen:
- **Min-Max Normalization** asegura que los datos estén dentro de un rango [0, 1], útil cuando es importante mantener los valores en ese rango.
- **Standardization (z-score)** ajusta los datos para que tengan una media de 0 y una desviación estándar de 1, y es útil cuando las características tienen unidades diferentes o distribuciones sesgadas.